In [4]:
from helper_methods import *
xml_root = get_xml_root('https://water.weather.gov/ahps2/hydrograph_to_xml.php?gage=rmdv2&output=xml')
forecast_data = get_flood_data(xml_root, 'forecast')
observed_data = get_flood_data(xml_root, 'observed')



In [ ]:
import plotly.subplots

make_subplots()